In [368]:
import numpy as np
from numpy import transpose
from numpy.linalg import inv, det
from scipy.stats import norm

In [34]:
def eta(T):
    """ Generates the cutoff probabilities for exploration rounds in interval chaining. """
    return np.array([pow(t, -1/3) for t in range(1,T+1)])

In [257]:
def beta(k, d, c):
    """ Generates the scaled down feature weights for a true model. """
    return np.random.uniform(0, c+1, size=(k, d))

In [502]:
def top_interval(c, k, d, _delta, T):
    """
    Simulates T rounds of interval chaining.
    """
    X = np.random.uniform(0, 1, size=(k, T, d))  # 3-axis ndarray
#     X = np.random.normal(0.5, 0.5, (k, T, d))
    _eta = eta(T)                             # exploration cutoff probabilities
    B = beta(k, d, c)                         # true parameters. B[i]: params for arm i
    Y = np.array([X[i].dot(transpose(B[i])) for i in range(k)])  # not sure if there's a cleaner way to do this
    picks = []
    for t in range(T):
        print('Iteration [{0} / {1}]'.format(t, T))
        r = np.random.rand()
        if r <= _eta[t]:
            # Play uniformly at random from [1, k].
            picks.append(np.random.randint(0,k))
            print('Exploration round.')
        else:
            intervals = []
            for i in range(k):
                # Compute beta hat.
                _Xti = X[i][:t+1]
                _XtiT = transpose(_Xti)
                try:
                    _XTX = inv(_XtiT.dot(_Xti))
                except:
                    print('Encountered singular matrix. Ignoring.')
                    continue
                _Yti = Y[i][:t+1]
                Bh_t_i = _XTX.dot(_XtiT).dot(_Yti)  # Compute OLS estimators.
                yh_t_i = Bh_t_i.dot(X[i][t])
                _s2 = np.var(Y[i][:t+1])
                w_t_i = norm.ppf(1 - _delta/(2*T*k), loc=0, scale=np.sqrt(_s2 * X[i][t].dot(_XTX).dot(transpose(X[i][t]))))
                intervals.append([yh_t_i - w_t_i, yh_t_i + w_t_i])
            # Pick the agent with the largest upper bound.
            picks.append(np.argmax(np.array(intervals)[:,1]) if intervals else np.random.randint(0,k))
            print('Intervals: {0}'.format(intervals))
    # Compute sum of best picks over each iteration.
    best = [transpose(Y)[i].max() for i in range(2, T)]
    performance = [Y[picks[t-2]][t] for t in range(2, T)]
#     print(picks)
#     print('Best: {0}'.format(sum(best)))
#     print('Algorithm: {0}'.format(sum(performance)))
    print('Cumulative Regret: {0}'.format(sum(best) - sum(performance)))
    print('Final Regret: {0}'.format(best[-1] - performance[-1]))
            

In [503]:
top_interval(c=10, k=2, d=10, _delta=0.05, T=1000)

Iteration [0 / 1000]
Exploration round.
Iteration [1 / 1000]
Intervals: [[87.355234894829465, 169.35000095060477], [nan, nan]]
Iteration [2 / 1000]
Exploration round.
Iteration [3 / 1000]
Exploration round.
Iteration [4 / 1000]
Exploration round.
Iteration [5 / 1000]
Exploration round.
Iteration [6 / 1000]
Encountered singular matrix. Ignoring.
Intervals: [[-7.8582029236726729, 20.749270248666534]]
Iteration [7 / 1000]
Exploration round.
Iteration [8 / 1000]
Exploration round.
Iteration [9 / 1000]
Intervals: [[10.135364548746708, 52.122100595795125], [9.8194982179500592, 44.864273060734419]]
Iteration [10 / 1000]
Exploration round.
Iteration [11 / 1000]
Exploration round.
Iteration [12 / 1000]
Exploration round.
Iteration [13 / 1000]
Intervals: [[6.5368756576614331, 40.828223365238117], [-1.7085715508536268, 39.371282356241984]]
Iteration [14 / 1000]
Exploration round.
Iteration [15 / 1000]
Exploration round.
Iteration [16 / 1000]
Intervals: [[14.184480413274418, 46.333911445043455], [

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:33: RuntimeWarning: invalid value encountered in sqrt



Iteration [218 / 1000]
Exploration round.
Iteration [219 / 1000]
Exploration round.
Iteration [220 / 1000]
Intervals: [[22.1897077473146, 32.071119344218467], [7.6478365184227011, 16.353760593630174]]
Iteration [221 / 1000]
Intervals: [[24.061359833653071, 33.820232651988633], [19.214685749757411, 27.93440680421914]]
Iteration [222 / 1000]
Exploration round.
Iteration [223 / 1000]
Exploration round.
Iteration [224 / 1000]
Exploration round.
Iteration [225 / 1000]
Intervals: [[17.938465926426296, 26.426443495726062], [11.051214374369286, 18.844220145506746]]
Iteration [226 / 1000]
Intervals: [[24.450083457773861, 35.320507110955866], [16.671942581383842, 24.426916931261072]]
Iteration [227 / 1000]
Intervals: [[26.023683489810853, 35.118344200250988], [8.1694004053680089, 17.522241387349034]]
Iteration [228 / 1000]
Intervals: [[14.685030681361763, 25.031303565143926], [15.378325680634246, 22.980307643527198]]
Iteration [229 / 1000]
Intervals: [[29.189882421396781, 38.448365653925045], [

### Testing

In [359]:
tmp = np.array([[ 0.41617394,  0.54593447,  0.10577638,  0.66642839,  0.21852587,
         0.12408503,  0.19740174,  0.16025822,  0.34112945,  0.79106745],
       [ 0.09355889,  0.68958286,  0.77347255,  0.02070936,  0.07084052,
         0.5991455 ,  0.45762994,  0.50857438,  0.17216335,  0.11820663]])

In [362]:
transpose(tmp).dot(tmp)

array([[ 0.18195401,  0.29172031,  0.11638661,  0.27928767,  0.09757253,
         0.10769634,  0.12496881,  0.11427695,  0.1580766 ,  0.34028094],
       [ 0.29172031,  0.77356897,  0.59112039,  0.37810705,  0.16815121,
         0.48090276,  0.42334218,  0.43819466,  0.30495522,  0.51338426],
       [ 0.11638661,  0.59112039,  0.60944843,  0.0865105 ,  0.07790807,
         0.47654786,  0.37484464,  0.41031986,  0.16924706,  0.17510583],
       [ 0.27928767,  0.37810705,  0.0865105 ,  0.44455568,  0.14709891,
         0.09510171,  0.14103135,  0.11733288,  0.23090374,  0.52963779],
       [ 0.09757253,  0.16815121,  0.07790807,  0.14709891,  0.05277194,
         0.06955957,  0.07555613,  0.07104824,  0.08674175,  0.18124252],
       [ 0.10769634,  0.48090276,  0.47654786,  0.09510171,  0.06955957,
         0.37437242,  0.29868152,  0.3245957 ,  0.14547995,  0.1689826 ],
       [ 0.12496881,  0.42334218,  0.37484464,  0.14103135,  0.07555613,
         0.29868152,  0.24839261,  0.26437411

In [191]:
k = 5
d = 10
T = 50
B = beta(k, d)
X = np.random.uniform(size=(k, T, d))
Y = np.array([X[i].dot(transpose(B[i])) for i in range(k)])

In [156]:
s2 = np.var(Y[0])
s2

0.54880669730882059

In [192]:
xTx = transpose(X[0][:10]).dot(X[0][:10])
Bh = inv(xTx).dot(transpose(X[0][:10])).dot(Y[0][:10])

In [152]:
Bh.dot(X[0][9])

2.9963963502844737

In [153]:
B[0].dot(X[0][9])

2.9963963502845434

In [90]:
#X[0].dot(np.transpose(sample_b[0]))

In [280]:
np.random.normal(0.5, 0.2)

0.691813602964692

In [438]:
sum([1,2])

3

In [498]:
[1,2,3][-1]

3